In [1]:
from dataset import load_dataset
from models.model_3 import ClimatePINN
from train import plot_comparison
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
import xarray as xr
from visualisation import visualize_predictions

In [2]:
temp = load_dataset(
    1, 
    train_val_split = None, 
    year0=1979, 
    root_dir="./data/era_5_data",
    normalize=True)
load = DataLoader(temp['train'], batch_size=16, shuffle=True)
data = next(iter(load))
inputs, masks, targets, coords = data['input'], data['masks'], data['target'], data['coords']

In [3]:
model = ClimatePINN()
a = model.forward(inputs, masks, coords)

In [4]:
a['physics_loss']

{'e1': tensor(0.0068, grad_fn=<MseLossBackward0>),
 'e2': tensor(0.0090, grad_fn=<MseLossBackward0>),
 'e3': tensor(0.0091, grad_fn=<MseLossBackward0>)}

In [5]:
u = a['output'][:,0,...]
v = a['output'][:,1,...]
Re = model.reynolds_network(u,v)

In [6]:
# Assuming Re is your tensor
Re_array = Re.cpu().detach().numpy()[0, 0, ...]

# Normalize the data for visualization
normalized_Re = (Re_array - np.min(Re_array)) / (np.max(Re_array) - np.min(Re_array))

# Plot the normalized data
plt.imshow(normalized_Re, cmap='viridis')  

tensor(50631.5508, grad_fn=<MeanBackward0>)

In [7]:

# # Example usage
# checkpoint_path = "checkpoints/run_1.pt"  # Update with your checkpoint path
# data_dir = "./data/era_5_data"  # Update with your data directory
# visualize_predictions(checkpoint_path, data_dir,72)